In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [3]:
from sklearn.manifold import TSNE

def tsne_plot(x1, y1, name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)
#     plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y1 == 0), 0], X_t[np.where(y1 == 0), 1], marker='o', color='g', linewidth='1', alpha=0.8, label='Non Fraud', s=2)
    plt.scatter(X_t[np.where(y1 == 1), 0], X_t[np.where(y1 == 1), 1], marker='o', color='r', linewidth='1', alpha=0.8, label='Fraud', s=2)

    plt.legend(loc='best');
    plt.savefig(name);
    plt.show();

In [4]:
df = pd.read_csv("./Data/sensor.csv", index_col=0)

In [5]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

In [6]:
df = df.drop('sensor_15', axis=1)
df = df.fillna(method='ffill')

In [7]:
df_broken = df[df['machine_status'] =='BROKEN']
df_sensors = df.drop(['machine_status'], axis=1)

In [8]:
# data = df_sensors.append(df_broken).sample(frac=1).reset_index(drop=True)
# X = data.drop(['machine_status'], axis = 1).values
# Y = data["machine_status"].values

# tsne_plot(X, Y, "original.png")

In [12]:
N_sensor = int(df_broken.shape[1]-1)

In [13]:
timesteps = 24
inputs = tf.keras.Input(shape=(timesteps, N_sensor))

X = tf.keras.layers.Conv1D(filters = 16, kernel_size = 2, padding = 'same')(inputs)
X = tf.keras.layers.Dropout(rate = 0.25)(X)
X = tf.keras.layers.Flatten()(X)
X = tf.keras.layers.Dense(24)
X = tf.keras